In [4]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np
import random

In [15]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return int(c * r)

In [16]:
df = pd.read_csv("Coimbatore.csv")

In [17]:
df.head()

,Latitude,Longitude,Cluster
0,37.580335,-121.240076,0
1,37.604987,-121.040026,0
2,37.626297,-121.269253,0
3,37.559299,-121.154000,0
4,37.604199,-121.191619,0


In [18]:
df = df.drop(columns=['Cluster'])

In [19]:
df = df[:50]

In [20]:
df.head()

,Latitude,Longitude
0,37.580335,-121.240076
1,37.604987,-121.040026
2,37.626297,-121.269253
3,37.559299,-121.154000
4,37.604199,-121.191619


In [21]:
from scipy.spatial.distance import cdist

# Calculate the mean latitude and longitude
centroid_lat = df['Latitude'].mean()
centroid_lon = df['Longitude'].mean()

# Calculate distances from each point to the centroid
distances = cdist(df[['Latitude', 'Longitude']], [(centroid_lat, centroid_lon)])

# Find the index of the closest point to the centroid
centroid_index = distances.argmin()

# Get the coordinates of the centroid data point
centroid_point = df.iloc[centroid_index]

print("Centroid Latitude:", centroid_point['Latitude'])
print("Centroid Longitude:", centroid_point['Longitude'])

Centroid Latitude: 37.600023451254536
Centroid Longitude: -121.22390154391026


In [22]:
distances = pdist(df[['Latitude', 'Longitude']], 
                  lambda u, v: haversine(u[0], u[1], v[0], v[1]))

In [23]:
distance_matrix = squareform(distances)

In [24]:
distance_matrix

array([[ 0., 22.,  4., ...,  3.,  0.,  5.],
       [22.,  0., 25., ..., 26., 21., 16.],
       [ 4., 25.,  0., ...,  3.,  4.,  8.],
       ...,
       [ 3., 26.,  3., ...,  0.,  4.,  9.],
       [ 0., 21.,  4., ...,  4.,  0.,  4.],
       [ 5., 16.,  8., ...,  9.,  4.,  0.]])

In [25]:
def distance_to_time(distance, traffic_factor):
    """
    Convert distance to time based on traffic factor.
    Assume time = distance / speed, where speed is inversely proportional to traffic factor.
    """
    speed = np.random.uniform(low=20, high=60) / traffic_factor
    
    return (distance / speed)*60

traffic_factor_matrix = np.random.uniform(low=1, high=2, size=distance_matrix.shape)

time_matrix = distance_to_time(distance_matrix, traffic_factor_matrix)

In [26]:
time_matrix

array([[ 0.        , 42.92268809, 12.58968075, ...,  8.1084842 ,
         0.        , 10.96961472],
       [70.81541385,  0.        , 43.93510096, ..., 55.88312269,
        40.89940866, 53.84214136],
       [ 7.05188225, 57.25856565,  0.        , ...,  7.84319008,
        13.65208201, 25.13922031],
       ...,
       [ 9.09523999, 51.77575346,  7.9263697 , ...,  0.        ,
         7.23129956, 17.94695488],
       [ 0.        , 41.47145499,  8.21110892, ..., 10.16249973,
         0.        , 12.8683312 ],
       [13.48399039, 52.21140144, 21.86227679, ..., 20.58487305,
         7.459617  ,  0.        ]])

In [28]:
def generate_orders(num_points, num_kitchens, max_orders_per_customer=2, order_probability=0.5):
    kitchen_indices = np.random.choice(num_points, num_kitchens, replace=False)
    
    all_indices = set(range(num_points))
    customer_indices = list(all_indices - set(kitchen_indices))
    
    orders = []
    for i, customer_index in enumerate(customer_indices):
        if np.random.rand() < order_probability:
            num_orders = np.random.randint(1, max_orders_per_customer + 1)
            for j in range(num_orders):
                kitchen_index = np.random.choice(kitchen_indices)
                prep_time = np.random.uniform(10, 46)  # 10-45 mins
                orders.append({'Order Number': len(orders) + 1, 'Prep Time': prep_time, 'Kitchen': kitchen_index, 'Customer': customer_index})
    
    df_orders = pd.DataFrame(orders)
    return df_orders

In [29]:
num_points = len(time_matrix)  # Number of data points
num_kitchens = 6  # Number of kitchens

# Generate orders
df_orders = generate_orders(num_points, num_kitchens, max_orders_per_customer=5, order_probability=1)

In [30]:
df_orders['Customer'].value_counts().sort_values()

Customer
27    1
31    1
38    1
25    1
10    1
8     1
19    1
47    1
5     1
17    2
29    2
9     2
45    2
15    2
43    3
41    3
39    3
6     3
24    3
40    3
11    3
35    3
7     3
20    3
18    3
22    4
28    4
49    4
12    4
34    4
42    5
46    5
26    5
44    5
0     5
1     5
30    5
37    5
2     5
4     5
13    5
14    5
23    5
16    5
Name: count, dtype: int64

# VRP start

In [31]:
depot = centroid_index

In [32]:
centroid_index

35

# CVRP - Capacitated Vehicle Routeing Problem

In [33]:
vehicle_capacity = 3 # 3 orders.
num_vehicles_needed = 8

customer_demands = df_orders['Customer'].value_counts().reset_index()
customer_demands.columns = ['Customer', 'Demands']

demands = np.zeros(num_points, dtype=int)
demands[customer_demands['Customer']] = customer_demands['Demands']

# VRPPD - Vehicle Routeing Problem with Pickup and Drop

In [34]:
pickup_deliveries = df_orders[['Kitchen', 'Customer']].values.tolist()

In [35]:
max(time_matrix[0])

42.92268809109699

In [36]:
df_orders.head()

,Order Number,Prep Time,Kitchen,Customer
0,1,21.310781,36,0
1,2,21.237859,36,0
2,3,30.447253,36,0
3,4,14.372810,48,0
4,5,27.801476,21,0


# VRPTW - Vehicle Routeing Problem with Time Window

In [37]:
centroid_index

35

In [38]:
def generate_time_windows(pickup_delivery_list, service_cities, service_time=10000):
    time_windows = [(0, service_time) if city_idx in service_cities else (0, 0) for city_idx in range(max(pickup_delivery_list)[1] + 1)]
    return time_windows

In [39]:
service_cities = set(city_idx for _, city_idx in pickup_deliveries)
time_windows = generate_time_windows(pickup_deliveries, service_cities)

In [40]:
time_windows[centroid_index] = [0, 10000]

In [41]:
prep_times_map = {}

# Iterate over each order and store the prep times in the hashmap
for _, row in df_orders.iterrows():
    kitchen, customer, prep_time = row['Kitchen'], row['Customer'], row['Prep Time']
    prep_times_map[(kitchen, customer)] = prep_time

In [42]:
# Iterate over the hashmap and update the time matrix
for (kitchen, customer), prep_time in prep_times_map.items():
    estimated_arrival_time = max(time_matrix[int(customer)][int(kitchen)], prep_time)
    time_matrix[int(customer)][int(kitchen)] = max(estimated_arrival_time, time_matrix[int(customer)][int(kitchen)]*60)

In [43]:
time_matrix

array([[   0.        ,   42.92268809,   12.58968075, ...,    8.1084842 ,
          14.37280959,   10.96961472],
       [  70.81541385,    0.        ,   43.93510096, ...,   55.88312269,
        2453.96451964,   53.84214136],
       [   7.05188225,   57.25856565,    0.        , ...,    7.84319008,
          13.65208201,   25.13922031],
       ...,
       [   9.09523999,   51.77575346,    7.9263697 , ...,    0.        ,
           7.23129956,   17.94695488],
       [   0.        ,   41.47145499,    8.21110892, ...,   10.16249973,
           0.        ,   12.8683312 ],
       [  13.48399039,   52.21140144,   21.86227679, ...,   20.58487305,
         447.57701983,    0.        ]])

# Google-ORTools solver for our modelled VRP

In [44]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import sys

In [57]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    neo_dist_matrix = []

    for arr in time_matrix:
        row = []
        for val in arr:
            row.append(int(val))
        neo_dist_matrix.append(row)

    data["distance_matrix"] = neo_dist_matrix
    data["num_vehicles"] = num_vehicles_needed
    data["depot"] = int(depot)
    data["pickups_deliveries"] = pickup_deliveries
    
    return data

In [58]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        total_distance += route_distance
    print(f"Total Distance of all routes: {total_distance}m")

In [59]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Define Transportation Requests.
    for request in data["pickups_deliveries"]:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index)
        )
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index)
            <= distance_dimension.CumulVar(delivery_index)
        )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

In [60]:
if __name__ == "__main__":
    main()